In [28]:
class opt():
    hparamDatasetPath = r"/Users/nikolai/Downloads/UPC/VSC/Project/dataset/sequences",
    hparamYamlConfigPath = "/Users/nikolai/Downloads/UPC/VSC/Project/Workspace-AD-LiDAR/F0_Visualization/semantic-kitti-api/config/semantic-kitti.yaml",
    hparamNumberOfRandomPoints = 4000
    hparamDatasetSequence = '04'    
    hparamNumberOfClasses = 4
    hparamClassChoice = 'bicycle'
    hparamBatchSize = 32
    hparamNumberOfEpochs = 250 #TODO: add to config ?
    hparamNumberOfWorkers = 4 #TODO: add to config ?
    hparamOutputFolder = 'output' #TODO: add to config ?
    hparamDeviceType = 'cuda'
    hparamFeatureTransform = False

In [30]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from B0_Dataset.dataset import SemanticKittiDataset
from D0_Modeling.model import SegmentationPointNet
from B1_Dataloader.dataloader import DataLoader_
from A0_Configuration.hyperparam import Parsing
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

blue = lambda x: '\033[94m' + x + '\033[0m'

torch.manual_seed(123)

train_dataset = SemanticKittiDataset(
    data_catalog_path=opt.hparamDatasetPath[0],
    sequence_number=opt.hparamDatasetSequence,
    yaml_config_path=opt.hparamYamlConfigPath[0],
    n_points=opt.hparamNumberOfRandomPoints,
    class_choice=[opt.hparamClassChoice],
    action_type='train')

val_dataset = SemanticKittiDataset(
    data_catalog_path=opt.hparamDatasetPath[0],
    sequence_number=opt.hparamDatasetSequence,
    yaml_config_path=opt.hparamYamlConfigPath[0],
    n_points=opt.hparamNumberOfRandomPoints,
    class_choice=[opt.hparamClassChoice],
    action_type='val')

test_dataset = SemanticKittiDataset(
    data_catalog_path=opt.hparamDatasetPath[0],
    sequence_number=opt.hparamDatasetSequence,
    yaml_config_path=opt.hparamYamlConfigPath[0],
    n_points=opt.hparamNumberOfRandomPoints,
    class_choice=[opt.hparamClassChoice],
    action_type='test')

#y = next(iter(dataset))  

#print('--------------')
#print('Single file data: shape: ', y[0].shape)     
#print('Single file data: sample: ', y[0][0])     
#print('Single file labels: shape: ', y[1].shape) 
#print('Single file labels: sample: ', y[1][0])         
#print('--------------')

train_dataloader = DataLoader_(
    dataset = train_dataset,
    batch_size=opt.hparamBatchSize,
    shuffle=True,
    num_workers=int(opt.hparamNumberOfWorkers))

val_dataloader = DataLoader_(
    dataset = val_dataset,
    batch_size=opt.hparamBatchSize,
    shuffle=True,
    num_workers=int(opt.hparamNumberOfWorkers))

test_dataloader = DataLoader_(
    dataset = test_dataset,
    batch_size=opt.hparamBatchSize,
    shuffle=True,
    num_workers=int(opt.hparamNumberOfWorkers))

print(len(train_dataset), len(val_dataset), len(test_dataset))
# num_classes = len(dataset.classes)
# print('classes', num_classes)

try:
    os.makedirs(opt.hparamOutputFolder)
except OSError:
    pass

TypeError: __init__() got an unexpected keyword argument 'class_choice'

In [16]:
num_classes=opt.hparamNumberOfClasses
feature_transform=opt.hparamFeatureTransform
classifier = SegmentationPointNet(num_classes, feature_transform)

#if opt.model != '':
#    classifier.load_state_dict(torch.load(opt.model))


optimizer = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
classifier = classifier.to(opt.hparamDeviceType)
#classifier.cuda()

num_batch = len(train_dataset) / opt.hparamBatchSize

for epoch in range(opt.hparamNumberOfEpochs):
    scheduler.step()
    for i, data in enumerate(train_dataloader, 0):
        points, target = data
        points = points.transpose(2, 1)
        points, target = points.to(opt.hparamDeviceType), target.to(opt.hparamDeviceType)
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, trans, trans_feat = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1, 1)[:, 0] - 1
        #print(pred.size(), target.size())
        loss = F.nll_loss(pred, target)
        if opt.hparamFeatureTransform:
            loss += feature_transform(trans_feat) * 0.001
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' % (epoch, i, num_batch, loss.item(), correct.item()/float(opt.hparamBatchSize * 2500)))

        if i % 10 == 0:
            j, data = next(enumerate(test_dataloader, 0))
            points, target = data
            points = points.transpose(2, 1)
            points, target = points.to(opt.hparamDeviceType), target.to(opt.hparamDeviceType)
            classifier = classifier.eval()
            pred, _, _ = classifier(points)
            pred = pred.view(-1, num_classes)
            target = target.view(-1, 1)[:, 0] - 1
            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' % (epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.hparamBatchSize * 2500)))

    torch.save(classifier.state_dict(), '%s/seg_model_%s_%d.pth' % (opt.hparamOutputFolder, opt.hparamClassChoice, epoch))

## benchmark mIOU
shape_ious = []
for i,data in tqdm(enumerate(test_dataloader, 0)):
    points, target = data
    points = points.transpose(2, 1)
    points, target = points.to(opt.hparamDeviceType), target.to(opt.hparamDeviceType)
    classifier = classifier.eval()
    pred, _, _ = classifier(points)
    pred_choice = pred.data.max(2)[1]

    pred_np = pred_choice.cpu().data.numpy()
    target_np = target.cpu().data.numpy() - 1

    for shape_idx in range(target_np.shape[0]):
        parts = range(num_classes)#np.unique(target_np[shape_idx])
        part_ious = []
        for part in parts:
            I = np.sum(np.logical_and(pred_np[shape_idx] == part, target_np[shape_idx] == part))
            U = np.sum(np.logical_or(pred_np[shape_idx] == part, target_np[shape_idx] == part))
            if U == 0:
                iou = 1 #If the union of groundtruth and prediction points is empty, then count part IoU as 1
            else:
                iou = I / float(U)
            part_ious.append(iou)
        shape_ious.append(np.mean(part_ious))

print("mIOU for class {}: {}".format(opt.hparamClassChoice, np.mean(shape_ious)))

/Users/nikolai/opt/miniconda3/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


AttributeError: type object 'opt' has no attribute 'model'

In [7]:
train_dataloader.__class__

B1_Dataloader.dataloader.DataLoader_